In [29]:
import numpy as np 
import pandas as pd 
import requests
from sqlalchemy import create_engine

In [78]:
import numpy as np 
import pandas as pd 
from sqlalchemy import create_engine
import json

def connect_to_database(): 
    host="georgia-election-data.cohfciey6luf.us-east-2.rds.amazonaws.com"
    port=3306
    dbname="election_data"
    user="ayush"
    password="electme20214eva"
    conn = create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'
                .format(user, password, host, dbname)).connect()
    
    return conn

def lambda_handler(event, context):
    conn = connect_to_database()
    
    if 'queryStringParameters' in event: 
        try: 
            year = int(event['queryStringParameters']["year"])
        except KeyError: 
            year = 2016
            
        try: 
            month = int(event['queryStringParameters']["month"]) 
        except KeyError: 
            month = 11
            
        try: 
            sheet_number = int(event['queryStringParameters']["sheet_number"])
        except KeyError: 
            sheet_number = 2
        
        try:
            result_column = event['queryStringParameters']["result_column"]
        except KeyError:
            result_column = "total_votes"
    else: 
        year = 2016 
        month = 11 
        sheet_number = 2
        result_column="total_votes"
    
    #event validation
    assert year > 1990 & year < 2022 
    assert month > 0 & month < 13
    assert sheet_number > 1 & sheet_number < 350
    assert result_column in ["election_day", "provisional", "absentee_by_mail", "advance_in_person", "total_votes"]
    
    query = """WITH cands AS (SELECT candidate_id, 
        co.contest_id, 
        candidate_name, 
        candidate_party
    FROM 
    contests co JOIN candidates ca ON co.contest_id = ca.contest_id
    WHERE co.sheet_number = {0} AND co.election_year = {1} AND ca.election_month = {2})
    
    SELECT 
        candidate_name,
        candidate_party,
        county_name, 
        {3} 
    FROM results_by_county rbc JOIN cands cad ON rbc.candidate_id = cad.candidate_id
    ORDER BY candidate_name, county_name;""".format(sheet_number, year, month, result_column)
    
    total_results = pd.read_sql_query(query, conn)
    pivoted_results = pd.DataFrame({"county_name": total_results.county_name.unique()})

    for candidate in total_results.candidate_name.unique():
        pivoted_results[candidate.strip()] = total_results[total_results.candidate_name == candidate].reset_index()[result_column]
    
    conn.close()
    
    return {
        'statusCode': 200,
        'body': pivoted_results.to_json()
    }

In [90]:
params = {"year": 2016, "month": 11, "result_column": "total_votes", "sheet_number": 3}
response = requests.get("https://qp4b96543m.execute-api.us-east-2.amazonaws.com/active/get_results", params=params).text
pd.read_json(response)

,county_name,ALLEN BUCKLEY,JIM BARKSDALE,JOHNNY ISAKSON (I)
0,Appling,154,1187,5170
1,Atkinson,65,596,1678
2,Bacon,95,474,3046
3,Baker,29,531,763
4,Baldwin,464,6787,7796
...,...,...,...,...
154,Whitfield,1381,6763,19808
155,Wilcox,70,669,2036
156,Wilkes,99,1525,2507
157,Wilkinson,79,1616,2266


In [133]:
import numpy as np 
import pandas as pd 
from sqlalchemy import create_engine
import json

def connect_to_database(): 
    host=""
    port=3306
    dbname="election_data"
    user="ayush"
    password=""
    conn = create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'
                .format(user, password, host, dbname)).connect()
    
    return conn

def lambda_handler(event):
    """
    Returns voter turnout by county on zero to three conditions across gender, race, and/or 
    age group. Currently only supports single selection (i.e. single race or all). 
    """
    if 'queryStringParameters' in event: 
        params = event['queryStringParameters']
        gender = params["gender"] if "gender" in params else None 
        race = params["race"] if "race" in params else None 
        age_grp = params["age_grp"] if "age_grp" in params else None
    else: 
        gender = None
        race = None
        age_grp = None
        
    #input validation 
    assert gender in [None, "MALE", "FEMALE"]
    assert race in [None, "BLACK", "WHITE", "ASIA-PI", "NATIVE-AM", "HISP-LT"]
    assert age_grp in [None, "18-24", "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59", "60-64", "65-OVER"]
    
    where_condn = ""
    if gender or race or age_grp:
        where_condn = "WHERE "
        gender_condn = f"gender = '{gender}'" if gender is not None else None
        age_condn = f"age_grp = '{age_grp}'" if age_grp is not None else None
        race_condn = f"race = '{race}'" if race is not None else None
        where_condn += (" AND ".join([cond for cond in [gender_condn, age_condn, race_condn] if cond]))
    
    query = """SELECT county_name, SUM(voted)/SUM(registered) AS turnout FROM county_turnout 
                {0}
                GROUP BY county_name 
                ORDER BY county_name;""".format(where_condn)
    
    conn = connect_to_database()
    turnout = pd.read_sql_query(query, conn)
    conn.close()
    
    return {
        'statusCode': 200,
        'body': turnout.to_json()
    }
    
    
params = {"race": "WHITE", "age_grp": "18-24"}
event = {}
lambda_handler(event)

{'statusCode': 200,
 'body': '{"county_name":{"0":"Appling","1":"Atkinson","2":"Bacon","3":"Baker","4":"Baldwin","5":"Banks","6":"Barrow","7":"Bartow","8":"Ben Hill","9":"Berrien","10":"Bibb","11":"Bleckley","12":"Brantley","13":"Brooks","14":"Bryan","15":"Bulloch","16":"Burke","17":"Butts","18":"Calhoun","19":"Camden","20":"Candler","21":"Carroll","22":"Catoosa","23":"Charlton","24":"Chatham","25":"Chattahoochee","26":"Chattooga","27":"Cherokee","28":"Clarke","29":"Clay","30":"Clayton","31":"Clinch","32":"Cobb","33":"Coffee","34":"Colquitt","35":"Columbia","36":"Cook","37":"Coweta","38":"Crawford","39":"Crisp","40":"Dade","41":"Dawson","42":"Decatur","43":"DeKalb","44":"Dodge","45":"Dooly","46":"Dougherty","47":"Douglas","48":"Early","49":"Echols","50":"Effingham","51":"Elbert","52":"Emanuel","53":"Evans","54":"Fannin","55":"Fayette","56":"Floyd","57":"Forsyth","58":"Franklin","59":"Fulton","60":"Gilmer","61":"Glascock","62":"Glynn","63":"Gordon","64":"Grady","65":"Greene","66":"Gwinn

In [143]:
import numpy as np 
import pandas as pd 
from sqlalchemy import create_engine
import json

def connect_to_database(): 
    host="georgia-election-data.cohfciey6luf.us-east-2.rds.amazonaws.com"
    port=3306
    dbname="election_data"
    user="ayush"
    password="electme20214eva"
    conn = create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'
                .format(user, password, host, dbname)).connect()
    
    return conn

def lambda_handler(event):
    """
    Returns voter turnout on a single county on a single axis. 
    """
    if 'queryStringParameters' in event and event['queryStringParameters'] is not None:      
        try: 
            year = int(event['queryStringParameters']["year"])
        except KeyError: 
            year = 2016

        try: 
            month = int(event['queryStringParameters']["month"]) 
        except KeyError: 
            month = 11
        try: 
            county = event['queryStringParameters']["county_name"]
        except KeyError: 
            county = 'Fulton'
        try: 
            axis = event['queryStringParameters']["axis"]
        except KeyError: 
            axis = 'age_grp' 
        try: 
            metric = event['queryStringParameters']["metric"]
        except KeyError: 
            metric = "voted"
    else: 
        year = 2016
        month = 11
        county = 'Fulton'
        axis = 'age_grp'
        metric = 'voted'

    metrics =  ["voted", "registered", "turnout"]

    #input validation 
    assert county in ["Appling","Atkinson","Bacon","Baker","Baldwin","Banks","Barrow","Bartow","Ben Hill",
        "Berrien","Bibb","Bleckley","Brantley","Brooks","Bryan","Bulloch","Burke","Butts","Calhoun","Camden",
        "Candler","Carroll","Catoosa","Charlton","Chatham","Chattahoochee","Chattooga","Cherokee","Clarke","Clay",
        "Clayton","Clinch","Cobb","Coffee","Colquitt","Columbia","Cook","Coweta","Crawford","Crisp","Dade","Dawson",
        "Decatur","DeKalb","Dodge","Dooly","Dougherty","Douglas","Early","Echols","Effingham","Elbert","Emanuel",
        "Evans","Fannin","Fayette","Floyd","Forsyth","Franklin","Fulton","Gilmer","Glascock","Glynn","Gordon",
        "Grady","Greene","Gwinnett","Habersham","Hall","Hancock","Haralson","Harris","Hart","Heard","Henry","Houston",
        "Irwin","Jackson","Jasper","Jeff Davis","Jefferson","Jenkins","Johnson","Jones","Lamar","Lanier","Laurens",
        "Lee","Liberty","Lincoln","Long","Lowndes","Lumpkin","Macon","Madison","Marion","McDuffie","McIntosh","Meriwether",
        "Miller","Mitchell","Monroe","Montgomery","Morgan","Murray","Muscogee","Newton","Oconee","Oglethorpe","Paulding",
        "Peach","Pickens","Pierce","Pike","Polk","Pulaski","Putnam","Quitman","Rabun","Randolph","Richmond","Rockdale",
        "Schley","Screven","Seminole","Spalding","Stephens","Stewart","Sumter","Talbot","Taliaferro","Tattnall","Taylor",
        "Telfair","Terrell","Thomas","Tift","Toombs","Towns","Treutlen","Troup","Turner","Twiggs","Union","Upson","Walker",
        "Walton","Ware","Warren","Washington","Wayne","Webster","Wheeler","White","Whitfield","Wilcox","Wilkes","Wilkinson",
        "Worth"]
    assert year > 1990 and year < 2022
    assert month > 0 and month < 13 
    assert axis in ["age_grp", "gender", "race"]
    assert metric in metrics
    
    metric_strings = ["SUM(voted)", "SUM(registered)", "SUM(voted)/SUM(registered)"]
    
    query = """SELECT {0}, {2} AS {3}
                FROM county_turnout
                WHERE county_name = '{1}'
                GROUP BY {0} 
                ORDER BY {0}; 
                    """.format(axis, county, metric_strings[metrics.index(metric)], metric)
    
    conn = connect_to_database()
    turnout = pd.read_sql_query(query, conn)
    conn.close()
    
    return {
        'statusCode': 200,
        'body': turnout.to_json()
    }

    
params = {"county_name": "Appling", "axis": "gender", "metric": "voted"}
event = {'queryStringParameters': params}
lambda_handler(event)

{'statusCode': 200,
 'body': '{"gender":{"0":"FEMALE","1":"MALE","2":"UNKNOWN"},"voted":{"0":3925.0,"1":3184.0,"2":0.0}}'}

In [136]:
import numpy as np 
import pandas as pd 
from sqlalchemy import create_engine
import json

def connect_to_database(): 
    host="georgia-election-data.cohfciey6luf.us-east-2.rds.amazonaws.com"
    port=3306
    dbname="election_data"
    user="ayush"
    password="electme20214eva"
    conn = create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'
                .format(user, password, host, dbname)).connect()
    
    return conn

def lambda_handler(event):
    """
    Returns voter turnout by county on zero to three conditions across gender, race, and/or 
    age group. Currently only supports single selection (i.e. single race or all). 
    """
    if 'queryStringParameters' in event and event['queryStringParameters'] is not None:      
        try: 
            year = int(event['queryStringParameters']["year"])
        except KeyError: 
            year = 2016
        try: 
            month = int(event['queryStringParameters']["month"]) 
        except KeyError: 
            month = 11
        try: 
            county = event['queryStringParameters']["county_name"]
        except KeyError: 
            county = 'Fulton'
        try axis = event['queryStringParameters']["county_name"]
    else: 
        year = 2016
        month = 11
        county = 'Fulton'

        
    #input validation 
    assert county in ["Appling","Atkinson","Bacon","Baker","Baldwin","Banks","Barrow","Bartow","Ben Hill",
        "Berrien","Bibb","Bleckley","Brantley","Brooks","Bryan","Bulloch","Burke","Butts","Calhoun","Camden",
        "Candler","Carroll","Catoosa","Charlton","Chatham","Chattahoochee","Chattooga","Cherokee","Clarke","Clay",
        "Clayton","Clinch","Cobb","Coffee","Colquitt","Columbia","Cook","Coweta","Crawford","Crisp","Dade","Dawson",
        "Decatur","DeKalb","Dodge","Dooly","Dougherty","Douglas","Early","Echols","Effingham","Elbert","Emanuel",
        "Evans","Fannin","Fayette","Floyd","Forsyth","Franklin","Fulton","Gilmer","Glascock","Glynn","Gordon",
        "Grady","Greene","Gwinnett","Habersham","Hall","Hancock","Haralson","Harris","Hart","Heard","Henry","Houston",
        "Irwin","Jackson","Jasper","Jeff Davis","Jefferson","Jenkins","Johnson","Jones","Lamar","Lanier","Laurens",
        "Lee","Liberty","Lincoln","Long","Lowndes","Lumpkin","Macon","Madison","Marion","McDuffie","McIntosh","Meriwether",
        "Miller","Mitchell","Monroe","Montgomery","Morgan","Murray","Muscogee","Newton","Oconee","Oglethorpe","Paulding",
        "Peach","Pickens","Pierce","Pike","Polk","Pulaski","Putnam","Quitman","Rabun","Randolph","Richmond","Rockdale",
        "Schley","Screven","Seminole","Spalding","Stephens","Stewart","Sumter","Talbot","Taliaferro","Tattnall","Taylor",
        "Telfair","Terrell","Thomas","Tift","Toombs","Towns","Treutlen","Troup","Turner","Twiggs","Union","Upson","Walker",
        "Walton","Ware","Warren","Washington","Wayne","Webster","Wheeler","White","Whitfield","Wilcox","Wilkes","Wilkinson",
        "Worth"]
    
    query = """SELECT co.contest_name, co.sheet_number, SUM(total_votes) AS total_votes
                FROM candidates ca 
                JOIN contests co ON ca.contest_id = co.contest_id
                JOIN results_by_county rbc ON ca.candidate_id = rbc.candidate_id
                WHERE county_name = '{0}' AND co.election_month = 11 AND co.election_year = 2016
                GROUP BY contest_name
                ORDER BY total_votes DESC;""".format(county)
    
    conn = connect_to_database()
    turnout = pd.read_sql_query(query, conn)
    conn.close()
    
    return {
        'statusCode': 200,
        'body': turnout.to_json()
    }


"Appling",
"Atkinson",
"Bacon",
"Baker",
"Baldwin",
"Banks",
"Barrow",
"Bartow",
"Ben Hill",
"Berrien",
"Bibb",
"Bleckley",
"Brantley",
"Brooks",
"Bryan",
"Bulloch",
"Burke",
"Butts",
"Calhoun",
"Camden",
"Candler",
"Carroll",
"Catoosa",
"Charlton",
"Chatham",
"Chattahoochee",
"Chattooga",
"Cherokee",
"Clarke",
"Clay",
"Clayton",
"Clinch",
"Cobb",
"Coffee",
"Colquitt",
"Columbia",
"Cook",
"Coweta",
"Crawford",
"Crisp",
"Dade",
"Dawson",
"Decatur",
"DeKalb",
"Dodge",
"Dooly",
"Dougherty",
"Douglas",
"Early",
"Echols",
"Effingham",
"Elbert",
"Emanuel",
"Evans",
"Fannin",
"Fayette",
"Floyd",
"Forsyth",
"Franklin",
"Fulton",
"Gilmer",
"Glascock",
"Glynn",
"Gordon",
"Grady",
"Greene",
"Gwinnett",
"Habersham",
"Hall",
"Hancock",
"Haralson",
"Harris",
"Hart",
"Heard",
"Henry",
"Houston",
"Irwin",
"Jackson",
"Jasper",
"Jeff Davis",
"Jefferson",
"Jenkins",
"Johnson",
"Jones",
"Lamar",
"Lanier",
"Laurens",
"Lee",
"Liberty",
"Lincoln",
"Long",
"Lowndes",
"Lumpkin",
"Macon",
"Madison",
"Marion",
"